# Milestone 1
## Introduction
< insert introduction text here >

## Background
< insert background text here > 

## How to Use autodiff32
### Installation

Create a virtual environment (optional):

1. From the terminal, create a virtual environment:

    `virtualenv env`

2. Activate the virtual environment: 

    `source env/bin/activate`

Install the autodiff32 package:

    `pip install autodiff32`

_(Package dependencies will be taken care of automatically)_



## Software Organization
### Directory Structure
Our structure is as follows:

    `/cs207-FinalProject`
        `README.md`

        `LICENSE`
        `.gitignore`
        `.travis.yml`
        `setup.py`
        `docs/`
            `milestone1.ipynb`
            `milestone2.ipynb`
        `autodif_32/`
            < TBD >
        `test/`
            < TBD >

< Insert description of each file and purpose >


### How to use

Start Python interpreter:

    `Python`

Import autodiff32:

    `>>> import autodiff32 as ad`

    `>>> import  numpy as np`

    
Initialize an AutoDiff object with the number you would like to pass into your function:

    `>>> X = ad.AutoDiff(3)`
    
Define your function of interest:

    `>>> func = 1/ad.sin(X**2)

    
Look at the derivative of your function evaluated at the number you gave:

    `>>> func.der`
    `>>> 32.187521164486526`

Look at the value of your function evaluated at the number you gave:

    `>>> func.val`
    `>>> 2.426486643551989`


